<a href="https://colab.research.google.com/github/joshtburdick/misc/blob/master/plog/EP_discrete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EP with discrete variables

Here, we try to implement EP with just discrete variables.

In [9]:
# prompt: import likely stuff for plotting using jax

import jax.nn as nn
import jax.numpy as jnp
import jax.random as jrandom
import jax.tree_util as tree_util
from jax import grad, jit, vmap
import matplotlib.pyplot as plt


## Moments
For converting between moments, we just use nn.softmax().

In [10]:
x = jnp.array([[1.,2.], [0.,3.]])

x


Array([[1., 2.],
       [0., 3.]], dtype=float32)

In [11]:
nn.softmax(x)

Array([[0.26894143, 0.7310586 ],
       [0.04742587, 0.95257413]], dtype=float32)

In [12]:
nn.softmax(x, axis=1)

Array([[0.26894143, 0.7310586 ],
       [0.04742587, 0.95257413]], dtype=float32)

Note that taking logs, and softmax, are (mostly) inverses.

In [13]:
y = nn.softmax(x)
print(y)
print(jnp.log(y))
print(nn.softmax(jnp.log(y)))

[[0.26894143 0.7310586 ]
 [0.04742587 0.95257413]]
[[-1.3132616  -0.31326166]
 [-3.0485873  -0.04858734]]
[[0.26894143 0.7310586 ]
 [0.04742587 0.95257413]]


# Implementing EP with discrete variables

This is (an attempt at) a limited case of EP:
- all variables are discrete, and take one of a set of values
- potentials are arrays (possibly sparse)
- messages are passed in parallel


## Defining the model, and initializing

The model will be represented using strings for variable and factor names; it will have a dict of variables, and a dict of factors.

The model state will include the current beliefs (as natural parameters), and the factor-to-variable messages.

In [14]:
# practice
rng = jrandom.key(42)

a = jrandom.uniform(rng, 4)
# (a / jnp.linalg.norm(a, 1))
r, rng = jrandom.split(rng)
b = jrandom.uniform(r, 4)
(a,b)

(Array([0.48870957, 0.6797972 , 0.6162715 , 0.5610161 ], dtype=float32),
 Array([0.5302608 , 0.31336212, 0.90153027, 0.6983329 ], dtype=float32))

### Sudoku example

For 4-by-4 Sudoku, the variables are a grid of variables with four possible values. We also need a 4-D tensor for the potential constraining all four numbers to be different. (This tensor will be re-used several times.)

This will also need potentials for filling in "givens".
(Hopefully, as a test, if we omit these, this will just generate
valid Sudoko boards?)


In [15]:
# prompt: Compute variables and factors which can be passed to ep_init (below) for 4-by-4 sudoku.

import numpy as np
# The state space for each variable is {0, 1, 2, 3}
variable_values = jnp.arange(4)

# The variables are arranged in a 4x4 grid
grid_size = 4
variables = {}
for i in range(grid_size):
  for j in range(grid_size):
    var_name = f'v_{i}_{j}'
    variables[var_name] = {'values': variable_values}

# The factors enforce the constraint that each row, column, and 2x2 block must contain each number from 0 to 3 exactly once.

# We need a tensor that represents the "all different" constraint for 4 variables.
# This tensor will have a 1 if all values are different, and 0 otherwise.
all_different_tensor = jnp.zeros((grid_size, grid_size, grid_size, grid_size))

# Generate all possible combinations of 4 numbers from 0 to 3
from itertools import product
for combo in product(range(grid_size), repeat=grid_size):
    if len(set(combo)) == grid_size: # Check if all values are different
        all_different_tensor = all_different_tensor.at[combo].set(1.0)

factors = {}

# Row factors
for i in range(grid_size):
  factor_name = f'row_factor_{i}'
  for j in range(grid_size):
    factor_vars = [f'v_{i}_{k}' for k in range(grid_size)]
    factors[factor_name] = {'variables': factor_vars, 'potential': all_different_tensor}

# Column factors
for j in range(grid_size):
  factor_name = f'col_factor_{j}'
  for i in range(grid_size):
    factor_vars = [f'v_{k}_{j}' for k in range(grid_size)]
    factors[factor_name] = {'variables': factor_vars, 'potential': all_different_tensor}

# 2x2 block factors
block_size = 2
for block_row in range(grid_size // block_size):
  for block_col in range(grid_size // block_size):
    factor_name = f'block_factor_{block_row}_{block_col}'
    factor_vars = []
    for i in range(block_size):
      for j in range(block_size):
        row_idx = block_row * block_size + i
        col_idx = block_col * block_size + j
        factor_vars.append(f'v_{row_idx}_{col_idx}')
    factors[factor_name] = {'variables': factor_vars, 'potential': all_different_tensor}

# Also need factors for any pre-filled cells (givens)
# For example, if cell (0,0) is given to be 2:
# factors['given_0_0'] = {'variables': ['v_0_0'], 'potential': jnp.array([0., 0., 1., 0.])} # Potential is a one-hot encoding

# ep_init expects a dict of variables and a dict of factors.
# The variable dict contains the initial beliefs (e.g., uniform) and messages.
# The factor dict contains the potential functions and connected variables.

# Initialize variable beliefs (natural parameters) as uniform
# For each variable, the initial belief is log(uniform) = log(1/4) = -log(4)
initial_var_beliefs = {}
for var_name in variables:
  initial_var_beliefs[var_name] = jnp.full(len(variable_values), -jnp.log(grid_size))

# Initialize messages (factor-to-variable)
# All messages start as uniform (log(1/4)), representing no information yet.
initial_messages = {}
for factor_name in factors:
  for var_name in factors[factor_name]['variables']:
    message_key = (factor_name, var_name)
    initial_messages[message_key] = jnp.full(len(variable_values), -jnp.log(grid_size))

ep_init_vars = initial_var_beliefs
ep_init_factors = factors
ep_init_messages = initial_messages

# You would then pass ep_init_vars, ep_init_factors, and ep_init_messages to ep_init.
# The ep_init function (not provided here) would typically store these and set up the initial state.
# The messages are usually part of the 'state' or 'params' rather than the initial model definition,
# but they are needed to initialize the iterative process.

In [16]:

def ep_init(rng, variables, factors, ):
  """Initializes the model."""
  beliefs = {}
  # beliefs are initialized to be random
  for (var, n_vals) in variables.items():
    r, rng = jrandom.split(rng)
    beliefs[var] = jrandom.uniform(r, (n_vals,))
    beliefs[var] = (beliefs[var] / jnp.sum(beliefs[var]))
    beliefs[var] = jnp.log(beliefs[var])
  # messages from factors are initially flat (I think?)
  factor_to_var_messages = {}
  for (name, factor) in factors.items():
    factor_to_var_messages[name] = {var: jnp.zeros(n_vals) for var in factor.variables}
  return {'beliefs': beliefs, 'factor_to_var_messages': factor_to_var_messages}


## Updating the state

We need a function which will do one round of message-passing. It will be a function from state to state.

Note that according to Gemini, "observations" are just additional
factors (typically only connected to one variable).

### Messages from a factor

This takes the incoming messages to a factor, and computes messages from it.


In [17]:
def marginalize(a, messages_in):
  """Given the input messsages, compute the marginal messages.

  a: the factor, as an n-dimensional tensor
  messages_in: a list of 1-dimensional tensors, one for each
    of the n variables connected to the factor
  Returns: a list of 1-dimensional tensors, one for each variable
    connected to the factor
  """
  n = len(messages_in)
  for i in range(n):
      f = jnp.expand_dims(jnp.softmax(messages_in[i], axis=0),
        [j for j in range(i)])
      print(f.shape)
      a *= f
  axes = set(range(n))
  messages_out = [jnp.log(jnp.sum(a, axis=tuple(axes-{i}))) for i in range(n)]
  return messages_out

Here is what Gemini generated, for updating state:

In [18]:
# prompt: Using the above definitions for what a (discrete variable) factor graph looks like, and a state (like what ep_init() returns), write a function which takes a factor graph, and a state, and does one round of parallel "loopy belief propagation" updates.

import numpy as np
def ep_update(fg, state):
  """Performs one round of parallel loopy belief propagation updates.

  Args:
    fg: A dictionary representing the factor graph. It should have keys
      'variables' and 'factors'. 'variables' is a dict mapping variable names
      to the number of values the variable can take. 'factors' is a dict
      mapping factor names to dictionaries with keys 'variables' (a list of
      variable names connected to the factor) and 'potential' (a JAX array
      representing the factor potential).
    state: A dictionary representing the current state. It should have keys
      'beliefs' and 'factor_to_var_messages'. 'beliefs' is a dict mapping
      variable names to JAX arrays of natural parameters.
      'factor_to_var_messages' is a dict mapping factor names to dictionaries,
      which map variable names to JAX arrays representing the factor-to-variable
      message natural parameters.

  Returns:
    A new state dictionary after one round of updates.
  """
  new_beliefs = state['beliefs'].copy()
  new_factor_to_var_messages = state['factor_to_var_messages'].copy()

  # Update variable-to-factor messages
  var_to_factor_messages = {}
  for var, belief in state['beliefs'].items():
    var_to_factor_messages[var] = {}
    for factor_name, factor_messages in state['factor_to_var_messages'].items():
      if var in fg['factors'][factor_name]['variables']:
        # Sum of all messages from other factors to this variable
        sum_other_messages = jnp.sum(jnp.stack([
            msg for other_factor_name, other_factor_messages in state['factor_to_var_messages'].items()
            for other_var, msg in other_factor_messages.items()
            if other_var == var and other_factor_name != factor_name
        ], axis=0), axis=0)
        var_to_factor_messages[var][factor_name] = belief - sum_other_messages

  # Update factor-to-variable messages
  for factor_name, factor_info in fg['factors'].items():
      for var in factor_info['variables']:
          # Sum of all incoming messages to the factor from other variables
          incoming_messages_sum = jnp.sum(jnp.stack([
              var_to_factor_messages[other_var][factor_name]
              for other_var in factor_info['variables'] if other_var != var
          ], axis=0), axis=0)

          # Expand incoming messages to the factor's dimension
          # This requires carefully handling the variable order and dimensions
          incoming_messages_expanded = incoming_messages_sum # This is too simple and needs to be generalized

          # For simplicity in this general definition, we'll skip the detailed marginalization step
          # which requires knowing the variable order within the factor potential.
          # A proper implementation would marginalize the product of the factor potential and
          # incoming messages over all variables except the one we are sending the message to.

          # Placeholder for the updated message - in a full implementation, this would involve
          # combining the factor potential and incoming messages, then marginalizing.
          # For this high-level update function structure, we'll represent the output message
          # as a placeholder that should result from this process.
          # The actual calculation depends heavily on the structure of the factor potential and
          # the variable order.

          # As a placeholder, let's imagine a process that results in an array of the
          # correct dimension for the variable `var`.
          # For now, we'll just keep the old message as a placeholder for this function's structure.
          # A complete EP implementation requires detailed handling of array dimensions and axis summation.

          # To make this runnable without a full marginalization implementation:
          # Let's compute a simplified update that represents the message as the belief
          # minus the incoming message from this factor. This is the message FROM the variable TO the factor.
          # The message FROM the factor TO the variable is more complex.

          # Let's refine the goal: perform the variable-to-factor message calculation accurately,
          # and represent the factor-to-variable message update step, even if the internal
          # calculation is simplified for this general structure.

          # The standard factor-to-variable message update:
          # 1. Combine the factor potential with the incoming messages from *other* variables connected to this factor.
          # 2. Marginalize the result over all variables in the factor except the one the message is being sent to.
          # 3. Subtract the incoming message from the variable being sent to from the result (or divide in moment form).

          # This simplified implementation will just show the message passing structure.

          # Re-calculating variable-to-factor messages (as needed for the factor update)
          var_to_factor_msgs_for_factor = {}
          for connected_var in factor_info['variables']:
              if connected_var != var:
                  sum_other_messages_to_connected_var = jnp.sum(jnp.stack([
                      msg for other_factor_name, other_factor_messages in state['factor_to_var_messages'].items()
                      for other_var, msg in other_factor_messages.items()
                      if other_var == connected_var and other_factor_name != factor_name
                  ], axis=0), axis=0)
                  var_to_factor_msgs_for_factor[connected_var] = state['beliefs'][connected_var] - sum_other_messages_to_connected_var


          # Placeholder for the factor-to-variable message update.
          # This part requires knowing the layout of the factor potential and
          # the variables it connects.
          # A full implementation would involve array manipulations (like broadcasting and `jnp.sum`)
          # based on the specific factor and variables.

          # For the purpose of showing the function structure, we'll represent the updated
          # message as a zero array of the correct size. This is NOT the correct EP update,
          # but keeps the code runnable and shows where the update would happen.
          num_values = fg['variables'][var]
          new_factor_to_var_messages[factor_name][var] = jnp.zeros(num_values) # Placeholder

  # Update beliefs
  for var in fg['variables'].keys():
      # Sum of all messages from factors to this variable
      sum_incoming_factor_messages = jnp.sum(jnp.stack([
          msg for factor_name, factor_messages in new_factor_to_var_messages.items()
          for connected_var, msg in factor_messages.items()
          if connected_var == var
      ], axis=0), axis=0)
      new_beliefs[var] = sum_incoming_factor_messages # In log space, beliefs are sum of log factor potentials (or messages)

  return {'beliefs': new_beliefs, 'factor_to_var_messages': new_factor_to_var_messages}


As it says, it includes placeholders, so presumably isn't the actual
correct implementation (although it does compile).

One difference: I'm assuming that the beliefs are passed into this function (and so have already been updated).

In [19]:
def ep_update(fg, state):
  """Performs one round of parallel loopy belief propagation updates.

  Args:
    fg: A dictionary representing the factor graph. It should have keys
      'variables' and 'factors'. 'variables' is a dict mapping variable names
      to the number of values the variable can take. 'factors' is a dict
      mapping factor names to dictionaries with keys 'variables' (a list of
      variable names connected to the factor) and 'potential' (a JAX array
      representing the factor potential).
    state: A dictionary representing the current state. It should have keys
      'beliefs' and 'factor_to_var_messages'. 'beliefs' is a dict mapping
      variable names to JAX arrays of natural parameters.
      'factor_to_var_messages' is a dict mapping factor names to dictionaries,
      which map variable names to JAX arrays representing the factor-to-variable
      message natural parameters.

  Returns:
    A new state dictionary after one round of updates.
  """
  # Compute variable-to-factor messages, by subtracting out
  # factor-to-variable messages from beliefs (and converting
  # from logs).
  var_to_factor_messages = {
      (v): nn.softmax(state['beliefs'][v] - state['factor_to_var_messages'][f][v])
      for f, v in state['factor_to_var_messages']
  }

  # Compute marginals, from each factor.
  def marginals(factor_name):
    m =

marginalize(fg["factors"][factor_name],

  # Update variable-to-factor messages
  var_to_factor_messages = {}
  for var, belief in state['beliefs'].items():
    var_to_factor_messages[var] = {}
    for factor_name, factor_messages in state['factor_to_var_messages'].items():
      if var in fg['factors'][factor_name]['variables']:
        # Sum of all messages from other factors to this variable
        sum_other_messages = jnp.sum(jnp.stack([
            msg for other_factor_name, other_factor_messages in state['factor_to_var_messages'].items()
            for other_var, msg in other_factor_messages.items()
            if other_var == var and other_factor_name != factor_name
        ], axis=0), axis=0)
        var_to_factor_messages[var][factor_name] = belief - sum_other_messages





SyntaxError: invalid syntax (ipython-input-19-1769059286.py, line 31)

In [ ]:
# practice
x = jnp.array([0., 3., 2., -5])
print(x)
print(x.shape)
print(nn.softmax(x))
print(nn.softmax(x).shape)
print(nn.softmax(x).sum())